ТЗ:

1. Возьми данные с meteo.ru/data по среднемесячным осадкам и температуре за июнь и июль с 2008 по 2019 г. (у меня лежат в папках temper_data и rain_data)
2. Отсюда можно взять координаты станций (на первом листе) и координаты географических центров (на втором листе) (файлы cords_stations и cords_regions)
https://vk.com/away.php?to=https://docs.google.com/sp 
3. Попробуй посчитать средние показатели для регионов, по метеостанциям, которые находятся в радиусе 500 км от их географического центра.
4. Если можно как-то сделать, чтобы этот радиус можно было менять, не залезая в код - было бы очень здорово.
5. Было бы здорово, если можно было бы менять координаты геоцентра на координаты столицы не влезая в код.

In [1]:
import pandas as pd
import numpy as np
import geopy.distance # какая-то узкоспециализированная библиотека чтоб считать расстояние зная широту и долготу

In [2]:
# задаём радиус
radius = 500

In [3]:
# Если мы хотим координаты ГЕОЦЕНТРА то True
# Если СТОЛИЦЫ то False
center = True

In [4]:
stations = pd.read_csv("cords_stations.csv")
# фрагмент 
stations.head()

,Широта,Долгота,Подпись,Номер метки,Код станции,Unnamed: 5,Код региона
0,"80,62","58,05","Им.Э.Т.Кренкеля, ГМО",1,20046,NaN,29
1,"79,50","76,98",Визе,2,20069,NaN,24
2,"79,55","90,62",Голомянный,3,20087,NaN,24
3,"78,07","14,25",Баренцбург,4,20107,Норвегия,0
4,"77,17","96,43",Русский,5,20289,NaN,24


In [5]:
# меняю широту и долготу в числовой формат
stations['Широта'] = stations['Широта'].replace(',','.', regex=True).values
stations['Долгота'] = stations['Долгота'].replace(',','.', regex=True).values
stations['Широта'] = stations['Широта'].astype(float)
stations['Долгота'] = stations['Долгота'].astype(float)

In [6]:
regions = pd.read_csv("cords_regions.csv")
# фрагмент 
regions.head()

,Код,Наименование,Широта столицы,Долгота столицы,Широта геоцентра,Долгота геоцентра
0,22,Алтайский край,53.3529,83.67371,51.79362,82.67585
1,28,Амурская область,50.27277,127.5404,54.6035,127.48017
2,29,Архангельская область,64.54725\n,40.56015,63.28528,42.58841
3,30,Астраханская область,46.3588,48.05993,46.13211,48.06101
4,31,Белгородская область,50.59971,36.59826,50.71069,37.75333


В таблице есть баг, совершенно рандомно стоит \n (знак перевода строки). Исправляю это

По каким то регионам нет данных и это выражено знаком -, заменяем на нан

In [7]:
regions = regions.replace('-', np.nan, regex=True)

In [8]:
regions = regions[['Код','Широта столицы','Долгота столицы','Широта геоцентра', 'Долгота геоцентра']].apply(pd.to_numeric)

In [9]:
regions.head() # баг с \n пропал

,Код,Широта столицы,Долгота столицы,Широта геоцентра,Долгота геоцентра
0,22,53.35290,83.67371,51.79362,82.67585
1,28,50.27277,127.54040,54.60350,127.48017
2,29,64.54725,40.56015,63.28528,42.58841
3,30,46.35880,48.05993,46.13211,48.06101
4,31,50.59971,36.59826,50.71069,37.75333


In [10]:
if center:
    regions = regions[["Код", "Широта геоцентра", "Долгота геоцентра"]]
    regions.columns = ["Код", "Широта", "Долгота"]
else:
    regions = regions[["Код", "Широта столицы", "Долгота столицы"]]
    regions.columns = ["Код", "Широта", "Долгота"]

In [11]:
regions

,Код,Широта,Долгота
0,22,51.79362,82.67585
1,28,54.60350,127.48017
2,29,63.28528,42.58841
3,30,46.13211,48.06101
4,31,50.71069,37.75333
...,...,...,...
81,20,43.40233,45.71874
82,21,56.13112,47.24576
83,87,65.62983,171.69521
84,89,66.06530,76.93451


Теперь берём температуру и осадки

In [12]:
temper = pd.read_fwf("temper_data\wr41508a1.txt", header = None)
# фрагмент 
temper.head()

,0,1,2,3
0,20046,2008,NaN,NaN
1,20046,2009,0.6,0.2
2,20046,2010,0.5,NaN
3,20046,2011,0.6,0.6
4,20046,2012,1.0,0.9


In [13]:
rain = pd.read_fwf("rain_data\wr41430.txt", header = None)
# фрагмент 
rain.head()

,0,1,2,3
0,20046,2008,NaN,NaN
1,20046,2009,33.6,10.6
2,20046,2010,8.4,NaN
3,20046,2011,11.8,6.9
4,20046,2012,20.1,16.1


Подготовили входные данные. Теперь для каждого региона надо найти станции в радиусе и взять среднее

In [14]:
stations.head()

,Широта,Долгота,Подпись,Номер метки,Код станции,Unnamed: 5,Код региона
0,80.62,58.05,"Им.Э.Т.Кренкеля, ГМО",1,20046,NaN,29
1,79.50,76.98,Визе,2,20069,NaN,24
2,79.55,90.62,Голомянный,3,20087,NaN,24
3,78.07,14.25,Баренцбург,4,20107,Норвегия,0
4,77.17,96.43,Русский,5,20289,NaN,24


In [15]:
regions.head()

,Код,Широта,Долгота
0,22,51.79362,82.67585
1,28,54.60350,127.48017
2,29,63.28528,42.58841
3,30,46.13211,48.06101
4,31,50.71069,37.75333


In [16]:
regions = regions.drop([14]) # потому что под 14 номером у нас байконур какой-то без координат

In [17]:
# Тест

In [18]:
def dist(i, j):
    return geopy.distance.distance([regions["Широта"][i], regions["Долгота"][i]], [stations["Широта"][j], stations["Долгота"][j]]).km


In [19]:
regions.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
            69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85],
           dtype='int64')

In [20]:
july_temp_mean_for_regions = pd.DataFrame(index = range(2008, 2019))
july_rain_mean_for_regions = pd.DataFrame(index = range(2008, 2019))
august_temp_mean_for_regions = pd.DataFrame(index = range(2008, 2019))
august_rain_mean_for_regions = pd.DataFrame(index = range(2008, 2019))


In [24]:
# ТЕСТ 

for i in regions.index:
    #num = regions["Код"][i]
    july_temp = pd.DataFrame(index = range(2008, 2019))
    july_rain = pd.DataFrame(index = range(2008, 2019))
    august_temp = pd.DataFrame(index = range(2008, 2019))
    august_rain = pd.DataFrame(index = range(2008, 2019))
    for j in stations.index:
        if dist(i, j) <= radius: #аккуратно, очень нестройный непонятный код
            print(regions["Код"][i], stations["Код станции"][j])
            print(dist(i, j))
            t = temper[temper[0] == stations["Код станции"][j]]
            t.index = t[1]
            july_temp = pd.concat([july_temp, t[[2]]], axis=1, sort=False)
            august_temp = pd.concat([august_temp, t[[3]]], axis=1, sort=False)
            
            r = rain[rain[0] == stations["Код станции"][j]]
            r.index = r[1]
            july_rain = pd.concat([july_rain, r[[2]]], axis=1, sort=False)
            august_rain = pd.concat([august_rain, r[[3]]], axis=1, sort=False)
    
    july_temp_mean_for_regions[i] = july_temp.mean(axis = 1)
    july_rain_mean_for_regions[i] = july_rain.mean(axis = 1)
    august_temp_mean_for_regions[i] = august_temp.mean(axis = 1)
    august_rain_mean_for_regions[i] = august_rain.mean(axis = 1)
    
    break
    
    
august_rain_mean_for_regions

22 29539
446.15639468607606
22 29612
485.9252760913
22 29638
346.19847096106463
22 29645
450.68303420765716
22 29822
244.60706852823108
22 29838
190.85779427597535
22 29849
349.94785696588315
22 29858
464.12168303194505
22 29915
303.72094541162
22 29923
145.06174959427983
22 29939
182.8567514033891
22 36034
104.7930778641666
22 36038
79.76801067010283
22 36045
115.76941865376484
22 36055
229.3600256637315
22 36064
339.7681102102991
22 36177
225.8990432642081
22 36229
267.1440951306719
22 36259
466.34003488410923
22 36442
330.0067759450465


,0
2008,50.926316
2009,38.869231
2010,34.450000
2011,52.910526
2012,49.421053
2013,65.526316
2014,50.677778
2015,52.182353
2016,41.311111
2017,51.442105


In [17]:
# аккуратно много хренового кода, работает долго

def dist(i, j):
    return geopy.distance.distance([regions["Широта"][i], regions["Долгота"][i]], [stations["Широта"][j], stations["Долгота"][j]]).km


july_temp_mean_for_regions = pd.DataFrame(index = range(2008, 2020))
july_rain_mean_for_regions = pd.DataFrame(index = range(2008, 2020))
august_temp_mean_for_regions = pd.DataFrame(index = range(2008, 2020))
august_rain_mean_for_regions = pd.DataFrame(index = range(2008, 2020))

for i in regions.index:
    num = regions["Код"][i]
    july_temp = pd.DataFrame(index = range(2008, 2020))
    july_rain = pd.DataFrame(index = range(2008, 2020))
    august_temp = pd.DataFrame(index = range(2008, 2020))
    august_rain = pd.DataFrame(index = range(2008, 2020))
    for j in stations.index:
        if dist(i, j) <= radius: #аккуратно, очень нестройный непонятный код
            
            t = temper[temper[0] == stations["Код станции"][j]]
            t.index = t[1]
            july_temp = pd.concat([july_temp, t[[2]]], axis=1, sort=False)
            august_temp = pd.concat([august_temp, t[[3]]], axis=1, sort=False)
            
            r = rain[rain[0] == stations["Код станции"][j]]
            r.index = r[1]
            july_rain = pd.concat([july_rain, r[[2]]], axis=1, sort=False)
            august_rain = pd.concat([august_rain, r[[3]]], axis=1, sort=False)
    
    july_temp_mean_for_regions[i] = july_temp.mean(axis = 1)
    july_rain_mean_for_regions[i] = july_rain.mean(axis = 1)
    august_temp_mean_for_regions[i] = august_temp.mean(axis = 1)
    august_rain_mean_for_regions[i] = august_rain.mean(axis = 1)
    
    
#august_rain_mean_for_regions

In [18]:
regions = pd.read_csv("cords_regions.csv") #считываю снова так как те столбцы удалены
regions = regions[["Код", "Наименование"]]
regions.head()

,Код,Наименование
0,22,Алтайский край
1,28,Амурская область
2,29,Архангельская область
3,30,Астраханская область
4,31,Белгородская область


In [19]:
def simple_things(a):
    a[14] = [np.nan] * 12
    a = (a.T).sort_index()
    a = pd.merge(regions, a, left_on=['Код'], right_index = True, how='left')
    return a

july_temp_mean_for_regions = simple_things(july_temp_mean_for_regions)
july_rain_mean_for_regions = simple_things(july_rain_mean_for_regions)

august_temp_mean_for_regions = simple_things(august_temp_mean_for_regions)
august_rain_mean_for_regions = simple_things(august_rain_mean_for_regions)
july_rain_mean_for_regions

,Код,Наименование,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,22,Алтайский край,68.796429,73.053571,17.135714,73.621429,87.053571,71.882143,49.628571,111.114286,55.850000,123.092857,73.335714,NaN
1,28,Амурская область,89.647826,90.130435,33.613043,76.256522,71.982609,79.717391,41.613043,95.030435,119.008696,117.021739,79.554167,NaN
2,29,Архангельская область,86.268750,60.434375,29.865625,72.196875,49.303030,79.966667,21.548485,68.754545,72.651515,90.445455,108.454545,NaN
3,30,Астраханская область,60.669231,31.676923,27.736364,47.692308,29.130769,72.766667,158.583333,33.866667,88.433333,93.475000,59.710000,NaN
4,31,Белгородская область,100.939394,75.421212,27.345455,83.272727,49.209091,89.436364,28.745455,85.118182,98.833333,116.606061,107.893939,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20,Чеченская Республика,69.393103,77.920690,51.537931,55.606897,61.603333,87.743333,71.437931,37.490000,73.541379,49.658621,97.436667,NaN
82,21,Чувашская Республика - Чувашия,74.984615,77.742105,105.557692,85.403846,45.130769,88.146154,85.330769,97.152000,108.824000,113.638462,77.011538,NaN
83,87,Чукотский автономный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,89,Ямало-Ненецкий автономный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
july_temp_mean_for_regions.to_excel('july_temp_mean_for_regions.xlsx', index = False, encoding = 'utf-8')
july_rain_mean_for_regions.to_excel('july_rain_mean_for_regions.xlsx', index = False, encoding = 'utf-8')
august_temp_mean_for_regions.to_excel('august_temp_mean_for_regions.xlsx', index = False, encoding = 'utf-8')
august_rain_mean_for_regions.to_excel('august_rain_mean_for_regions.xlsx', index = False, encoding = 'utf-8')

In [21]:
july_temp_mean_for_regions

,Код,Наименование,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,22,Алтайский край,19.271429,17.339286,22.453571,21.042857,19.410714,19.371429,16.367857,15.664286,20.407143,17.896429,20.250000,NaN
1,28,Амурская область,16.865217,17.073913,22.995652,21.043478,18.369565,17.643478,19.143478,15.647826,18.869565,15.917391,19.158333,NaN
2,29,Архангельская область,20.034375,20.690625,25.800000,23.171875,21.630303,19.539394,20.981818,19.603030,21.515152,19.290909,21.209091,NaN
3,30,Астраханская область,13.630769,13.976923,16.745455,15.492308,13.775000,14.241667,13.091667,15.216667,14.600000,12.950000,14.260000,NaN
4,31,Белгородская область,18.724242,18.772727,24.727273,22.090909,20.130303,18.496970,19.993939,17.787879,20.272727,17.469697,19.854545,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20,Чеченская Республика,22.468966,23.021429,24.827586,24.506897,22.976667,22.136667,22.972414,23.070000,23.065517,23.537931,24.730000,NaN
82,21,Чувашская Республика - Чувашия,19.096154,17.968421,16.796154,16.076923,20.234615,18.057692,18.750000,19.000000,19.624000,17.803846,17.238462,NaN
83,87,Чукотский автономный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,89,Ямало-Ненецкий автономный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
